In [ ]:
# install the latest VertexAI and langchain package
! pip install google-cloud-aiplatform google-cloud-discoveryengine langchain[docarray]==0.0.305 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0 --upgrade --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.7.0-cp310-cp310-linux_x86_64.whl size=2202681 sha256=ff8f94fb016fb3492a96762be853bff1fd1387eaf0137c28701222e2fcd52cf6
  Stored in directory: /root/.cache/pip/wheels/8a/ae/ec/235a682e0041fbaeee389843670581ec6c66872db856dfa9a4
Successfully built hnswlib


In [ ]:
import os
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

In [ ]:
PROJECT_ID = "cloud-sandbox-danielw"
SEARCH_ENGINE_ID = "unstructured-datastore_1695333435366"
REGION = "us-central1"
MODEL = "text-bison@001"

In [ ]:
# init the Vertext LLM model - bison
import vertexai
from langchain.llms import VertexAI

vertexai.init(project=PROJECT_ID, location=REGION)
bison = VertexAI(model_name=MODEL, temperature = 0.0)

In [ ]:
# Load the document data
from langchain.document_loaders import CSVLoader

file = '/content/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='utf-8')
data = loader.load()

In [ ]:
# Generate the Q&A pairs
from langchain.evaluation.qa import generate_chain

example_gen_chain = generate_chain.QAGenerateChain.from_llm(bison)

qa_pairs = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

# Print the generated question-answer pairs
for item in qa_pairs:
    print(f"Question: {item['qa_pairs']['query']}")
    print(f"Answer: {item['qa_pairs']['answer']}")

/root/.local/lib/python3.10/site-packages/langchain/chains/llm.py:308: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Question: What is the name of the shoe that is made of soft canvas?
Answer: Women's Campside Oxfords
Question: What is the name of the dog mat?
Answer: Recycled Waterhog Dog Mat, Chevron Weave
Question: What is the name of the toddler's two-piece swimsuit?
Answer: Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece
Question: What is the name of the swimwear item?
Answer: Refresh Swimwear, V-Neck Tankini Contrasts
Question: What is the name of the pants that are made of 100% nylon?
Answer: EcoFlex 3L Storm Pants


In [ ]:
examples = [
  {
    "qa_pairs": {
      "query": "Do the Cozy Comfort Pullover Set have side pockets?",
      "answer": "Yes"
    }
  },{
    "qa_pairs": {
        "query": "What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
  }
]

qa_pairs += examples

examples = []
for item in qa_pairs:
  examples.append({
    "query": item['qa_pairs']['query'],
    "answer": item['qa_pairs']['answer']
  })

In [ ]:
print(examples)

[{'query': 'What is the name of the shoe that is made of soft canvas?', 'answer': "Women's Campside Oxfords"}, {'query': 'What is the name of the dog mat?', 'answer': 'Recycled Waterhog Dog Mat, Chevron Weave'}, {'query': "What is the name of the toddler's two-piece swimsuit?", 'answer': "Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece"}, {'query': 'What is the name of the swimwear item?', 'answer': 'Refresh Swimwear, V-Neck Tankini Contrasts'}, {'query': 'What is the name of the pants that are made of 100% nylon?', 'answer': 'EcoFlex 3L Storm Pants'}, {'query': 'Do the Cozy Comfort Pullover Set have side pockets?', 'answer': 'Yes'}, {'query': 'What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?', 'answer': 'The DownTek collection'}]


In [ ]:
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import VertexAIEmbeddings

# create the in memory VectorDB index
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=VertexAIEmbeddings()
).from_loaders([loader])

# Q&A chain
qa = RetrievalQA.from_chain_type(
    llm=bison,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [ ]:
print(f"question: {examples[0]['qa_pairs']['query']}")
answer = qa.run(examples[0]['qa_pairs']["query"])
print(f"answer: {answer}")

question: Do the Cozy Comfort Pullover Set have side pockets?


> Entering new RetrievalQA chain...

> Finished chain.
answer: Yes


In [ ]:
import langchain
langchain.debug = True

qa.run(examples[0]['qa_pairs']["query"])

# Turn off the debug mode
langchain.debug = False

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set have side pockets?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set have side pockets?",
  "context": "Berber fleece-lined pockets keep hands warm. One pocket has a hidden security pocket tucked inside. Imported.<<<<>>>>>: 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyeste

In [ ]:
questions = []
for item in examples:
  questions.append({
    "query": item['query']
  })

predictions = qa.apply(questions)

print(f"\n{predictions}")



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.

[{'query': 'What is the name of the shoe that is made of soft canvas?', 'result': "I don't know the answer to this question."}, {'query': 'What is the name of the dog mat?', 'result': 'The name of the dog mat is Recycled Waterhog Dog Mat, Chevron Weave.'}, {'query': "What is the name of the toddler's two-piece swimsuit?", 'result': "Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece"}, {'query': 'What is the name of the swimwear item?', 'result': 'The name of the swimwear item is Mix-It-Up Tankini Print.'}, {'query': 'What is the name of the pants that are made of 100% nylon?', 'resul

In [ ]:
from langchain.evaluation.qa import QAEvalChain

eval_chain = QAEvalChain.from_llm(bison)

graded_outputs = eval_chain.evaluate(examples, predictions)

for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + examples[i]['query'])
    print("Real Answer: " + examples[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: What is the name of the shoe that is made of soft canvas?
Real Answer: Women's Campside Oxfords
Predicted Answer: I don't know the answer to this question.
Predicted Grade: INCORRECT

Example 1:
Question: What is the name of the dog mat?
Real Answer: Recycled Waterhog Dog Mat, Chevron Weave
Predicted Answer: The name of the dog mat is Recycled Waterhog Dog Mat, Chevron Weave.
Predicted Grade: CORRECT

Example 2:
Question: What is the name of the toddler's two-piece swimsuit?
Real Answer: Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece
Predicted Answer: Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece
Predicted Grade: CORRECT

Example 3:
Question: What is the name of the swimwear item?
Real Answer: Refresh Swimwear, V-Neck Tankini Contrasts
Predicted Answer: The name of the swimwear item is Mix-It-Up Tankini Print.
Predicted Grade: INCORRECT

Example 4:
Question: What is the name of the pants that are made of 100% nylon?
Real Answer: EcoFlex 3